# Task 3: Helper notebook for loading the data and saving the predictions

In [1]:
import pickle
import gzip
import numpy as np
import os
import cv2

### Helper functions

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [3]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

### Load data, make predictions and save prediction in correct format

In [4]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")

In [28]:
from typing import List, Tuple

def select_dataset(data:List[dict], quality:str) -> List[dict]:
    """Selects a dataset from the initial data

    Args:
        data (List[dict]): initial data (containing amateur and expert)
        quality (str): either 'amateur' or 'expert'

    Returns:
        List[dict]: dataset with only the quality chosen
    """
    return [dict_frames for dict_frames in data if dict_frames["dataset"]==quality]
    
def select_only_label_images(data:List[dict])->dict:
    """Retrieves only the images with labels from the dataset

    Args:
        data (List[dict]): output from select_dataset

    Returns:
        dict: key: name of the video | value: {"image":image, "label":label}
    """
    images_labelized = {}
    for video in data:
        indices_labelized_frames = video["frames"]
        for index in indices_labelized_frames:
            images_labelized[video["name"]+"_"+str(index)] = {"image":video["video"][:,:,index], "label":video["label"][:,:,index]}
    return images_labelized

def create_X_y(data_labelized:dict) -> Tuple[np.array, np.array]:
    """Creates two numpy arrays containing the images and labels

    Args:
        data_labelized (dict): output of select_only_label_images

    Returns:
        Tuple[np.array, np.array]: X, y
    """
    X, y = [], []
    for name in data_labelized.keys():
        X.append(data_labelized[name]["image"])
        y.append(data_labelized[name]["label"])
    return np.array(X), np.array(y)

### Split the Data and select either expert, amateur or both datasets

In [29]:
train_data_expert = select_dataset(train_data, "expert")
train_data_expert_labelized = select_only_label_images(train_data_expert)
X_train_expert, y_train_expert = create_X_y(train_data_expert_labelized)

/var/folders/rg/xgkyjs4945d6ckms1nn6v0jm0000gn/T/ipykernel_2386/751181434.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(X), np.array(y)


### Plot the data and its label overlaid

In [64]:
def labelize_image(image:np.array, label:np.array) -> np.array:
    """Labelize image (put the Mitral Valve detected fron label in white)

    Args:
        image (np.array): image
        label (np.array): label

    Returns:
        np.array: labelized image
    """
    labelized_image = image.copy()
    label = label.astype(np.uint8)  #convert to an unsigned byte
    label*=255
    labelized_image[label == 255] = 255
    cv2.imshow("labelized", labelized_image)
    cv2.waitKey(0)
    return 

In [65]:
image, label = X_train_expert[1], y_train_expert[1]
labelized_image = labelize_image(image, label)

113

### Visualize one given image

In [7]:
def visualize_and_resize(image:np.array, size:int, time:int):
    """Visualize an image for a certain time (useful to visualize the video afterwards)

    Args:
        image (np.array): image or frame from a given video
        size (int): size to display the image
        time (int): time in ms to display the image
    """
    up_points = (size, size)
    resized_up = cv2.resize(image, up_points, interpolation=cv2.INTER_LINEAR)
    cv2.imshow("Image", resized_up)
    cv2.waitKey(time)

In [8]:
first_video = train_data[0]
video = first_video["video"]
first_image = video[:,:,0]
visualize_and_resize(first_image, 600, 0)


### Save Predictions

In [9]:
# make prediction for test
predictions = []
for d in test_data:
    prediction = np.array(np.zeros_like(d['video']), dtype=np.bool)
    height = prediction.shape[0]
    width = prediction.shape[1]
    prediction[int(height/2)-50:int(height/2+50), int(width/2)-50:int(width/2+50)] = True
    
    # DATA Strucure
    predictions.append({
        'name': d['name'],
        'prediction': prediction
        }
    )

/var/folders/rg/xgkyjs4945d6ckms1nn6v0jm0000gn/T/ipykernel_2386/701677749.py:4: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  prediction = np.array(np.zeros_like(d['video']), dtype=np.bool)


In [10]:
# save in correct format
save_zipped_pickle(predictions, 'my_predictions.pkl')